# BT2101 K-Means and Hierarchical Clustering

## 1 Goal

In this notebook, we will explore clustering method, an unsupervised learning method, including:
* K-Means Clustering
* Hierarchical Clustering

In [ ]:
# -*- coding:utf-8 -*-
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import division
from math import sqrt
%matplotlib inline

### 1.1 K-Means Clusting

K-Means Clustering steps:

1. Determine the number of k (i.e., number of clusters)
2. Randomly select k centroids (i.e. centers of clusters)
3. Assign each data point to its closest centroid
4. Recalculate the centroids as the average of all data points in a cluster
5. Assign data points to their closest centroids
6. Repeat Step 4 and 5 until the observations are not reassigned or the maximum number of iterations is reached

In [ ]:
# Load libraries and dataset
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Load data
iris = datasets.load_iris()
features = iris.data

# Standardize features
scaler = StandardScaler()
features_std = scaler.fit_transform(features)

In [ ]:
# Fit k-means clustering model: Suppose 3 clusters
cluster_3 = KMeans(n_clusters=3, random_state=0, n_jobs=-1)
model_3 = cluster_3.fit(features)
model_std_3 = cluster_3.fit(features_std)

In [ ]:
# View cluster centers for each cluster:
model_std_3.cluster_centers_

In [ ]:
# View predicted clusters:
model_std_3.labels_

In [ ]:
# Compare with true classes:
iris.target

In [ ]:
# Predict which cluster is a new observation in:
new_obs = [[0.5, 0.5, 0.5, 0.5]]
model_std_3.predict(new_obs)

### How to choose k? 
#### Elbow method
* Gauge how the heterogeneity within clusters changes for various of k.
* The heterogeneity within clusters is expected to decreases with more clusters.
* The heterogeneity is measured by within-clusters/groups sum of squares (WSS)

\begin{align}
WSS(k) = \mathop{\sum}_{i=1}^{N}\mathop{\sum}_{j=0}^{p}(x_{ij}-centroid(x_{kj}))^2
\end{align}

Suppose N observations and p features. <br/>
k is cluster id ($=1,...,K$). <br/>
$x_{ij}$ is $i^{th}$ observation $j^{th}$ feature. <br/>
$centroid(x_{kj})$ is $k^{th}$ centroid of feature $j$.

In [ ]:
# Import scipy used for calculate distances
from scipy.spatial.distance import cdist

In [ ]:
# Cluster data with 1-10 clusters and get centroids

N_cluster = range(1, 11) # 1 to 10 clusters
WSS_list = [] # A list of WSS for 1 to 10 clusters

for k in N_cluster:
    
    # Run k-means model
    cluster = KMeans(n_clusters=k, random_state=0, n_jobs=-1) # Run the model: Assume k clusters
    model = cluster.fit(features_std) # Fit the standardized data
    centroids = model.cluster_centers_ # Get centroids for each cluster id (=0,...,k-1)
    labels = model.labels_ # Get labels (cluster id) for each observation
    
    # Calculate WSS(k)
    squared_distance = cdist(features_std, centroids, 'sqeuclidean') # Calculate squared distance between observations and centroids
    min_distance_cluster_id = np.argmin(squared_distance, axis=1) # Find index of minimum squared distance for each observation
    min_distance = np.min(squared_distance, axis=1) # Find minimum squared distance for each observation
    WSS = np.sum(min_distance)
    
    WSS_list.append(WSS)   
    

In [ ]:
# Show WSS for 1-10 clusters
WSS_list

In [ ]:
# Plot the elbow curve
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(N_cluster, WSS_list, 'g*-')
ax.plot(N_cluster[1], WSS_list[1], marker='o', markersize=12, markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
ax.plot(N_cluster[2], WSS_list[2], marker='o', markersize=12, markeredgewidth=2, markeredgecolor='b', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Within-cluster Sum of Squares (WSS)')
plt.title('Elbow curve for K-means clustering')

plt.show()

Question: Which number of clusters do you prefer?

The original code can be found at http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html.

### 1.2 Hierarchical Clustering

Hierarchical Clustering Steps:

1. Start with n clusters (each record is its own cluster)
2. Merge two closest records into one cluster
3. At each successive step, the two clusters closest to each other are merged 

In [ ]:
# Load Libraries
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# Create meanshift object
HC = AgglomerativeClustering(n_clusters=3)
HC_model = HC.fit(features_std)

In [ ]:
# Show clusters
HC_model.labels_

In [ ]:
# Plot dendrogram for hierarchical clustering
from scipy.cluster.hierarchy import dendrogram, linkage
avg_link = linkage(features_std[1:20, :], method='average') # Linkage criterion: average distance
dendrogram(avg_link).items

Question: How to decide the number of clusters?

More information about hierarchical clustering can be found at: <br/>
[1] http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html <br/>
[2] https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html

## 2 References
[1] Raschka, S. (2015). Python machine learning. Packt Publishing Ltd. <br/>
[2] Chris Albon. (2018). Machine Learning with Python Cookbook. O'Reilly.